# Машинное обучение и майнинг данных

## 16/02/2017 Линейные модели, практика

# Логистическая регрессия

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

Рассмотрим данные по кредитованию.

Столбец с классом называется `y`.<br/> Значение $1$ соответствует классу клиентов банка, которым выдали кредит и они его успешно вернули.<br/> Значение $-1$ соответствует клиентам, невыполнившим свои кредитные обязанности. 

В банке хотят уметь определять по признакам `a1-a15`, сможет ли новый клиент вернуть кредит или нет? То есть нам надо обучить классификатор! *Обычно, в банках используют скор-карты, но процесс их построения тесно связан с логистической регрессией*

Загрузите данные и преобразуйте признаки `a1`, `a9`, `a10` и `a12` из строковых в числовые. В них только 2 возможных значения. Для этого можно использовать функцию DataFrame.replace() в `pandas` или самое обычное присваивание на соответствующих строках.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('crx.data')

df.head()

### Предобработка данных

#### Заполнение пропусков

Заполните пропуски в данных

In [ ]:
# Your Code Here

#### Преобразование признаков

В признаках `a6`, `a7` присутствуют "редкие" значение. Найдите их с помощью фунцкии `.value_counts()`  и объедините, присвоив им одно и то же значение, например `'Other'`.

In [ ]:
# Your code here

Выделите бинарные признаки `a1`, `a9`, `a10` и `a12` в матрицу `X_binary`

In [ ]:
# Your code here


На занятиях мы изучатли преобразование категориальных признаков с помощью комбинации `LabelEncoder` и `OneHotEncoder` или метода `pd.get_dummies`. <br\>

Есть еще один способ из библиотеки sklearn - `DictVectorizer`. Если хотите - можете разобраться, что делает код ниже, изучая промежуточные результаты каждой из команд.

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
dv = DictVectorizer(sparse=False)
dv.fit(df[['a5', 'a6', 'a7', 'a13']].T.to_dict().values())

X_cat = dv.transform(df[['a5', 'a6', 'a7', 'a13']].T.to_dict().values())

`X_cat` - эквивалентна последовательному применению `LabelEncoder` к каждому признаку `a5`, `a6`, `a7`, `a13` и использованию `OneHotEncoder`. Чтобы узнать, чему соответствует каждый столбец матрицы `X_cat` используйте команду `dv.feature_names_`

In [ ]:
# Your code here

Вы долнжы увидеть список с элементами вида <Признак>=<Значение>. Если в каком-то из случаев это не так, то
* Либо `DictVectorizer` посчитал, что признак - не категориальный
* Либо в признаке есть пропуски

С помощью метода  df[..].corr() убедитесь, что признаки `a2`, `a3`, `a8`, `a11`, `a14` и `a15` довольно слабо коррелируют друг с другом.

Нормализуйте количественные признаки `a2`, `a3`, `a8`, `a11`, `a14` и `a15` с помощью `StandartScaler` или вручную. Вы должны получить матрицу `X_real`.

In [ ]:
# Your code here


Используйте функцию `np.concatinate(..)` или `np.c[..]` чтобы сцепить матрицы `X_binary`, `X_cat` и `X_real`

В результате вы должны получить матрицу с преобразованными призанками `X` и вектор ответов `y`

In [ ]:
# Your Code Here

### Регуляризация

Помимо мультиколлинеарности логистическая регрессия так же может быть подвержена переобучению. Как правило, на деле это выражается в 
* Очень больших весах $|w_i|$ при признаках
* Неустойчивости решения (особенно в случае мультиколлинеарности, когда возникает линейная зависимость признаков и получается целое семейство равнозначных моделей)
* Большая разница в качестве на обучении и валидации

Регуляризация - это техника понижения "сложности" модели с минимальными последствиями для её качества.

В случае с логистичесткой регресии, сложность модели выражается в значениях весов $w_j$ при признаках. Больший вес означает большее влияние признака на результат. В таком случае, давайте добавил штрафное слагаемое в функцию оптимизации для логистической регресии. Самый распространенные из них это:

* Ridge regression
$$L(w) = - \left(\sum_i \log(\sigma(y^{(i)} \langle w, x^{(i)} \rangle)) + \frac{1}{C}\sum_j w_j^2\right) \rightarrow \min_w$$

* Lasso regression
$$L(w) = -\left(\sum_i \log(\sigma(y^{(i)} \langle w, x^{(i)} \rangle) + \frac{1}{C}\sum_j |w_j|\right) \rightarrow \min_w$$

$C$ - называется гиперпараметром регуляризации и он задается вручную. Выбирается он с помощью кросс-валидации. Чем больше $С$ - тем меньше влияние регуляризации.

Lasso regression называется так, потому что она осуществляет "отлов" признаков - т.е. незначимые признаки будут иметь нулевые веса в модели, в то время как в Ridge regression - веса будут постепенно падать у всех признаков.

<img src='http://webdancer.is-programmer.com/user_files/webdancer/Image/lasso.png'>

Однако использование регуляризации превращает логистическую регрессию в черный ящик - мы предоставляем оптимизационному методу решать за нас, какие признаки важны для модели.

Давайте сравним работу регуляризаторов. 

1. Разбейте данные на обучающую и контрольную выборки.
1. Для $C$ из набора np.logspace(-3, 3, 10) обучите LogisctigRegression c Lasso регуляризацией (`penalty='l1'`). На каждой итерации оцените качество (ROC-AUC) на контрольной выборке и запомните полученные коэффициенты модели
1. На одном графике выведите значение качества в зависимости от параметра `C` 
1. На другом графике для каждого признака выведите изменение коэффициента в модели в зависимости от параметра `C`
1. Для оптимальной на ваш взгляд настройки модели выведите 5 наиболие "важных" признаков и их коэффициенты
1. Проделайте тоже самое для Ridge регуляризации (`penalty='l2'`)

In [ ]:
# Your Code Here

## Tree embedding

Вот так (примерно) выглядит скор-карта на выдачу кредита

<img src='https://www.mathworks.com/help/finance/credit_score_card_overview.png'>

Каждый "признак" разбит на бины (bins) - значимые интервалы. Если значение признака попадает в интервал, то к скору заемщика прибавляется сооветвтвующая величина.

Таким образом формируется итоговый скор и далее принимается решение о выдаче кредита.

Оценка порого бинов для признаков - это отдельная наука. Скоррингисты используют так называемое IV (information value) для оценки качества разбиения признака на бины.

На прошлом занятии мы изучили с вами другой способ выявления порогов на признаках - деревья решений (случайный лес). Почему бы нам это не использовать

### Задание
* Разбейте выборку на 2 части (обучение и контроль)
* На половине обучающей выборке обучите случайный лес
* С помощью метода `rand_forest.apply(X)` можно получить матрицу размера $n \times t$, где $t$ - количество деревьев в лесу. В эту матрицу будет числом записан номер листа, в который попал объект
* One-Hot Encoding!
* Обучите логистическую регрессию на второй половине обучающей выборки
* Сравние roc-auc на обычной лог-регрессии, обычного случайного леса и их композиции

In [ ]:
# Your Code Here

# Линейная регрессия

Квадратичная ошибка достаточно чувствительна к выбросам. Давайте вернемся к нашим данным про автомобили и добавим туда выбросы.

Посмотрим, как поведет себя простая линейная регрессия.

In [ ]:
df_train = pd.read_csv('http://bit.ly/1gIQs6C')

In [ ]:
X_train = df_train.mileage.values.reshape(-1, 1)
y_train = df_train.price.values

n = y_train.shape[0]

In [ ]:
## Добавляем выбросы

In [ ]:
X_train = np.r_[X_train, [[250000+np.random.rand()*10000]]]
y_train = np.r_[y_train, 16000+np.random.randn()*1000]

In [ ]:
plt.scatter(X_train, y_train)

In [ ]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train[:n], y_train[:n])

model_ouliers = LinearRegression(fit_intercept=True)
model_ouliers.fit(X_train, y_train)

In [ ]:
x = np.linspace(0, max(X_train), 100).reshape(-1, 1)
y_hat = model.predict(x)
y_hat_outliers = model_ouliers.predict(x)

fig, ax = plt.subplots(1, 1, figsize=(10,5))
ax.scatter(X_train, y_train)

ax.plot(x, y_hat, c='red')
ax.plot(x, y_hat_outliers, c='green')

### RANSAC регрессия

Идея метода RANdom SAmple Consensus (RANSAC) заключается в многократном обучении модели на случайном наборе точек из исходных данных с последующим выбором лучшей модели.

То есть:
* Задаем функцию потерь
* Задаем порог $\theta$ для остатков при котором наблюдения начинают относится к выбросам
* Задаем правило останова

Шаги алгоритма следующие
1. Взять случайные K точек и обучить на них модель M
2. Сравнить ошибки на остальных точких с порогом $\theta$ и отнести к выбросам или внутренним точкам
3. Обучить модель на всех внутренних точках, оценить качество на внутренних точках
4. Повторить 1-3 пока не наступит правило останова. 
5. Вывод: модель с лучшим качеством

In [ ]:
from sklearn.linear_model import RANSACRegressor

In [ ]:
model_ransac = RANSACRegressor(LinearRegression())
model_ransac.fit(X_train, y_train)

In [ ]:
x = np.linspace(0, max(X_train), 100).reshape(-1, 1)
y_hat = model_ransac.predict(x)

fig, ax = plt.subplots(1, 1, figsize=(10,5))
ax.scatter(X_train, y_train)

ax.plot(x, y_hat, c='red')
ax.plot(x, y_hat_outliers, c='green')

### Robust Estimators

Идея робастных методов заключается во взвешивании остатков модели таким образом, чтобы большие значения вносили меньший вклад в оценку параметров.

Таким образом, вместо минимизации квадрата остатков $$ L(\beta_0,\beta_1,\dots) = \frac{1}{2n}\sum^{n}_{i=1}(\hat{y}^{(i)} - y^{(i)})^2$$
Будут минимизироваться взвешенные остатки $$ L_w(\beta_0,\beta_1,\dots) = \frac{1}{2n}\sum^{n}_{i=1}\rho(\hat{y}^{(i)} - y^{(i)}),$$
где $\rho(\cdot)$ - некоторая взвешивающая функция.

Для того, чтобы попробовать эти методы нужно будет устновить пакет `statsmodels` через `pip`

In [ ]:
!pip install statsmodels

In [ ]:
import statsmodels.api as sm

In [ ]:
c = 4.685
support = np.linspace(-3*c, 3*c, 1000)
tukey = sm.robust.norms.TukeyBiweight(c=c)
plt.plot(support, tukey(support))
plt.ylim(.1, -4.1)

In [ ]:
model_robust = sm.RLM(y_train, sm.add_constant(X_train), M=sm.robust.norms.TukeyBiweight())
model_robust = model_robust.fit()

In [ ]:
model_robust.summary()

In [ ]:
x = np.linspace(0, max(X_train), 100).reshape(-1, 1)
y_hat = model_robust.predict(sm.add_constant(x))

fig, ax = plt.subplots(1, 1, figsize=(10,5))
ax.scatter(X_train, y_train)

ax.plot(x, y_hat, c='red')
ax.plot(x, y_hat_outliers, c='green')

## Немного практики

Загрузите [данные](http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv) о лесных пожарах в некоторых областях парка Montesinho в Португалии.

Описание данных следующее:
1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: 'jan' to 'dec'
4. day - day of the week: 'mon' to 'sun'
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3
7. DC - DC index from the FWI system: 7.9 to 860.6
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40
12. rain - outside rain in mm/m2 : 0.0 to 6.4
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 

Описание индексов FFMC, DMC, CD, ISI приводится [здесь](http://cwfis.cfs.nrcan.gc.ca/background/summary/fwi)

Ваша задача - по данным признакам 1-12 предсказать признак 13, площадь области, которая подвергнется пожару.

Перед тем как приступать с модели, постройте гистрограмму площади пожара `area`. Что можно сказать о том, как распределены значения? Рассмотрите различные преобразования, например `log(area+1)` или `sqrt(area)` и выберите то, которое будет лучше использовать для предсказания (линейной моделью).

In [ ]:
##  Your Code Here

### Преобразование признаков

Выполните преобразование признаков, а именно:

* Нормализацию вещественных признаков с помощью z-score (x - x.mean())/x.std(). Нормализацию зависимой переменной `area` выполнять не надо.
* Преобразование номинальных признаков `month` и `day` в числовое представление.
* Имеет ли смысл преобразование для признаков `X` и `Y`? Если да - выполните его.


На выходе вы должны получить матрицы X_train и X_test с преобразованными признаками, а так же векторы ответов y_train и y_test.ь

In [ ]:
##  Your Code Here

### Обучение

Обучите две модели:

* Простую линейную регрессию `LinearRegression` (без регуляризации). Выберите такое подмножество признаков из X_train, чтобы избежать мультиколлинеарности.
* Любую другую понравившеюся вам модель.

Расчитайте предсказания для контролькой выборки. 

* Найдите среднюю абсолютную ошибку модели на контрольной выборке
* Постройте график зависимости ошибок $|(\text{area}) - (\text{predicted_area})|$ и значений признака $\text{area}$, где $\text{area}$ и $\text{predicted_area}$ - значения в исходной шкале, а не преобзованной после задания 2.1
* Какие значения модель предсказывает лучше?

In [ ]:
##  Your Code Here